In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sklearn as skl
import pandas as pd
import tensorflow as tf

C:\Users\esobieski\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\esobieski\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\esobieski\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\esobieski\anaconda3\envs\mlenv\lib\site-packages\ten

In [2]:
# Import our input dataset
charity_df = pd.read_csv("C:/Users/esobieski/Documents/Berkeley/Neural_Networks/charity_data.csv")
charity_df.tail(75)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34224,990087275,INTERNATIONAL ASSOCIATION OF SHEET METAL AIR R...,T5,CompanySponsored,C3000,ProductDev,Association,1,1M-5M,N,11432603,0
34225,990140474,UNITED STATES PONY CLUBS INC,T3,CompanySponsored,C1200,Preservation,Association,1,0,N,5000,1
34226,990165772,AMERICAN WATER WORKS ASSOCIATION,T3,CompanySponsored,C1800,Preservation,Association,1,100000-499999,N,223007,1
34227,990187594,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,1
34228,990187607,PTA HAWAII CONGRESS,T3,Independent,C1000,Preservation,Trust,1,25000-99999,N,17192,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


# Characterize Input Data

In [3]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [4]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [5]:
# SPECIAL_CONSIDERATIONS HAS 27 YES and 122 NO, KEEP AND TRANSFORM
print(charity_df['SPECIAL_CONSIDERATIONS'].value_counts()['Y'] )

27


In [6]:
# Delete columns:
charity_df.drop(['STATUS', 'EIN', 'NAME'], axis=1, inplace=True)
charity_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [7]:
# Generate our NEW categorical variable list
charity_cat1 = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [8]:
# Check the number of unique values in each column
charity_df[charity_cat1].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

# Prepare 2 columns (classification and application type) for bucketing

In [9]:
# Check the number of unique values in column APPLICATION_TYPE
app_type_counts= charity_df.APPLICATION_TYPE.value_counts()  
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
other_app_type = app_type_counts[app_type_counts <200]

# Bucketing Application Type Column values

In [11]:
# Determine which values to replace
replace_app = list(app_type_counts[app_type_counts <200].index)

# Replace in DataFrame
for APPLICATION_TYPE in replace_app:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(APPLICATION_TYPE,"Other")

# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

# Bucketing Classification Column values

In [12]:
# Check the number of unique values in column CLASSIFICATION
class_counts= charity_df.CLASSIFICATION.value_counts()  
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1900        1
C1236        1
C1283        1
C2600        1
C3700        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
other_class = class_counts[class_counts <200]

In [14]:
other_class

C4000    194
C5000    116
C1270    114
C2700    104
C2800     95
        ... 
C1900      1
C1236      1
C1283      1
C2600      1
C3700      1
Name: CLASSIFICATION, Length: 64, dtype: int64

In [15]:
# Bucketing  CLASSIFICATION
# Determine which values to replace

replace_app = list(class_counts[class_counts <200].index)

# Replace in DataFrame
for CLASSIFICATION in replace_app:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(CLASSIFICATION,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

#  One Hot Encoder, Standard Scaler

In [16]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat1]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat1)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat1,1)
charity_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Split our preprocessed data into our features and target ("Attrition") arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a Binary Classification Deep Neural Net

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer (Use sigmoid activation because predicting binary classification output - Attrition yes or no?)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 360       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________


# Compile and Train First Model with 2 layers, 8 neurons, 5 neurons, 25 Epochs

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
# Train the scaled model
fit_model = nn.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6005 - acc: 0.7034
Epoch 2/25
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5675 - acc: 0.7265
Epoch 3/25
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5580 - acc: 0.7280
Epoch 4/25
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5559 - acc: 0.7288
Epoch 5/25
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5547 - acc: 0.7293
Epoch 6/25
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5538 - acc: 0.7301
Epoch 7/25
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5531 - acc: 0.7307
Epoch 8/25
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5519 - acc: 0.7305
Epoch 9/25
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5513 - acc: 0.7317
Epoch 10/25
25724/25724 [==============================] - 1s 34us/sample - loss: 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5523 - acc: 0.7248
Loss: 0.5523049362129789, Accuracy: 0.724781334400177


# TRY MORE NEURONS (48 and 8) KEEPING 2 LAYERS

In [24]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer (Use sigmoid activation because predicting binary classification output - Attrition yes or no?)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 40)                1800      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 328       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 2,137
Trainable params: 2,137
Non-trainable params: 0
_________________________________________________________________


# Compile and Train with more neurons

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5894 - acc: 0.7145
Epoch 2/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5555 - acc: 0.7307
Epoch 3/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5519 - acc: 0.7294
Epoch 4/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5503 - acc: 0.7321
Epoch 5/50
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5487 - acc: 0.7329
Epoch 6/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5479 - acc: 0.7316
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5475 - acc: 0.7331
Epoch 8/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5468 - acc: 0.7331
Epoch 9/50
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5461 - acc: 0.7335
Epoch 10/50
25724/25724 [==============================] - 1s 35us/sample - loss: 

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5540 - acc: 0.7271
Loss: 0.5540025636962135, Accuracy: 0.7271137237548828


# TRY LESS LAYERS AND 48 NEURONS

In [28]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  48

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Output layer (Use sigmoid activation because predicting binary classification output - Attrition yes or no?)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 48)                2160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 49        
Total params: 2,209
Trainable params: 2,209
Non-trainable params: 0
_________________________________________________________________


# Compile and Train with Less Layers

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5796 - acc: 0.7170
Epoch 2/50
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5575 - acc: 0.72940s - loss: 0.565
Epoch 3/50
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5530 - acc: 0.7317
Epoch 4/50
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5521 - acc: 0.7317
Epoch 5/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5506 - acc: 0.7318
Epoch 6/50
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5489 - acc: 0.7322
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5487 - acc: 0.7312
Epoch 8/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5482 - acc: 0.7323
Epoch 9/50
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5476 - acc: 0.7320
Epoch 10/50
25724/25724 [==============================] - 1s 30us

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5544 - acc: 0.7248
Loss: 0.5543982881359735, Accuracy: 0.724781334400177


# Change Input Activation Function using First Model (above)

In [32]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer (Use sigmoid activation because predicting binary classification output - Attrition yes or no?)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 8)                 360       
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 6         
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6030 - acc: 0.6953
Epoch 2/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5665 - acc: 0.7272
Epoch 3/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5599 - acc: 0.7297
Epoch 4/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5563 - acc: 0.7314
Epoch 5/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5541 - acc: 0.7315
Epoch 6/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5526 - acc: 0.7327
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5518 - acc: 0.7325
Epoch 8/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5509 - acc: 0.7324
Epoch 9/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5503 - acc: 0.7336
Epoch 10/50
25724/25724 [==============================] - 1s 31us/sample - loss: 

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5526 - acc: 0.7258
Loss: 0.5525708128411986, Accuracy: 0.7258309125900269


# Train first model, 2 layers, 8 and 5 neurons, with 50 epochs vs 25 first time

In [36]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer (Use sigmoid activation because predicting binary classification output - Attrition yes or no?)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 8)                 360       
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 6         
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6235 - acc: 0.6807
Epoch 2/50
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5656 - acc: 0.7262
Epoch 3/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5600 - acc: 0.7283
Epoch 4/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5565 - acc: 0.7278
Epoch 5/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5551 - acc: 0.7294
Epoch 6/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5535 - acc: 0.7296
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5531 - acc: 0.7301
Epoch 8/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5519 - acc: 0.7303
Epoch 9/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5514 - acc: 0.7309
Epoch 10/50
25724/25724 [==============================] - 1s 31us/sample - loss: 

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5508 - acc: 0.7259
Loss: 0.5508177991094116, Accuracy: 0.7259474992752075
